In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
# make sure close previose spark esssion opened
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('sql') \
    .getOrCreate()

24/03/02 14:16:55 WARN Utils: Your hostname, Desktop-Gar resolves to a loopback address: 127.0.1.1; using 172.25.243.204 instead (on interface eth0)
24/03/02 14:16:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/02 14:16:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/02 14:17:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Creating the dataframes

In [3]:
df_green = spark.read.parquet('data/pq/green/2020/01')
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [4]:
df_green.show()

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2|2020-01-19 03:36:12|2020-01-19 03:39:32|                 N|         5|          92|         264|              1|         0.78|        7.0|  0.0|    0.0|      1.46

In [5]:
df_yellow = spark.read.parquet('data/pq/yellow/2020/01')
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [6]:
df_yellow.head(5)

[Row(VendorID=2, pickup_datetime=datetime.datetime(2020, 1, 8, 21, 48, 33), dropoff_datetime=datetime.datetime(2020, 1, 8, 21, 54, 37), passenger_count=1, trip_distance=None, RatecodeID=1, store_and_fwd_flag=None, PULocationID=142, DOLocationID=None, payment_type=None, fare_amount=7.0, extra=0.5, mta_tax=0.5, tip_amount=2.16, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=12.96, congestion_surcharge=None),
 Row(VendorID=1, pickup_datetime=datetime.datetime(2020, 1, 21, 18, 55, 2), dropoff_datetime=datetime.datetime(2020, 1, 21, 19, 11, 22), passenger_count=1, trip_distance=None, RatecodeID=1, store_and_fwd_flag=None, PULocationID=231, DOLocationID=None, payment_type=None, fare_amount=16.0, extra=3.5, mta_tax=0.5, tip_amount=0.0, tolls_amount=6.12, improvement_surcharge=0.3, total_amount=26.42, congestion_surcharge=None),
 Row(VendorID=2, pickup_datetime=datetime.datetime(2020, 1, 12, 22, 46), dropoff_datetime=datetime.datetime(2020, 1, 12, 22, 53, 41), passenger_count=5, tri

# Combined dataset

In [7]:
common_colums = []

yellow_columns = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_colums.append(col)

In [8]:
from pyspark.sql import functions as F

In [9]:
df_green_sel = df_green \
    .select(common_colums) \
    .withColumn('service_type', F.lit('green'))

df_yellow_sel = df_yellow \
    .select(common_colums) \
    .withColumn('service_type', F.lit('yellow'))

In [10]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [11]:
df_trips_data.groupBy('service_type').count().show()


+------------+-------+
|service_type|  count|
+------------+-------+
|       green| 447746|
|      yellow|6404912|
+------------+-------+



In [12]:
df_trips_data.show()

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|congestion_surcharge|service_type|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|       2|2020-01-19 03:36:12|2020-01-19 03:39:32|                 N|         5|          92|         264|              1|         0.78|        7.0|  0.0|    0.0|      1.46|         0.0|       

In [13]:
df_trips_data.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge',
 'service_type']

# SQL

In [14]:
# df_trips_data.registerTempTable('trips_data')
df_trips_data.createOrReplaceTempView('trips_data')

In [15]:
spark.sql("""
SELECT
    service_type,
    count(1)
FROM
    trips_data
GROUP BY 
    service_type
""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green|  447746|
|      yellow| 6404912|
+------------+--------+



In [16]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [17]:
df_result.show()

+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|revenue_monthly_congestion_surcharge|avg_montly_passenger_count|avg_montly_trip_distance|
+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|          35|2020-

In [18]:
df_result.coalesce(1).write.parquet('data/report/revenue/', mode='overwrite')


In [19]:
!ls -lh data/report/revenue/

total 40K
-rwxrwxrwx 1 root root   0 Mar  2 14:21 _SUCCESS
-rwxrwxrwx 1 root root 38K Mar  2 14:21 part-00000-8f925e43-5011-4a3b-89d2-18dcb61141f5-c000.snappy.parquet
